## Word2Vec

### Introdução

Em machine learning, o objetivo é empregar o poder computacional para resolver problemas do mundo real. Diferentemente dos humanos, que possuem percepção direta, os computadores operam fundamentalmente com números, convertidos em última instância para sequências de __0's__ e __1's__.

Atualmente, algoritmos processam diversos tipos de dados, incluindo números, imagens, áudios e textos. Para que um computador possa trabalhar com essas informações, é necessário encontrar maneiras de representá-las numericamente, permitindo sua manipulação. No contexto do texto, existem várias técnicas capazes de convertê-lo em informação numérica. Essas técnicas variam desde abordagens mais simples, como a contagem de palavras no modelo _Bag of Words_, até modelos mais sofisticados, como os _Transformers_. Este estudo se concentra na técnica __Word2Vec__.

O __Word2Vec__ consiste em representar palavras em um espaço vetorial de _n-dimensões_, de forma que essa representação capture seu significado semântico. Geralmente, palavras semanticamente relacionadas, como "mulher" e "garota", terão representações vetoriais próximas nesse espaço. Em contraste, a similaridade vetorial entre palavras menos relacionadas, como "mulher" e "homem", será menor. Essa lógica se aplica a outros exemplos de palavras como visualizado abaixo.
> Word2vec é um algoritmo para obter word embeddings treinando uma rede neural rasa (com apenas uma hidden layer) com duas arquiteturas possíveis: CBOW ou Skip-Gram. ([Word Embedding: fazendo o computador entender o significado das palavras](https://medium.com/turing-talks/word-embedding-fazendo-o-computador-entender-o-significado-das-palavras-92fe22745057))

![Representação Vetorial de Palavras em um Plano 3D](https://miro.medium.com/v2/resize:fit:868/0*Cgod6JuBcJyd9GVM)

### Arquiteturas do Word2Vec

Antes de iniciar a exploração, vou importar algumas bibliotecas necessárias que utilizarei para esse projeto (evitar importações desnecessárias ou repetitivas).

In [1]:
import os
import pandas as pd
from gensim.models import KeyedVectors

Além disso, vou criar algumas funções comuns que vou utilizar nesse notebook python

In [5]:
import requests
import zipfile

# Funtion to download zip file from URL
def download_file_zip(URL: str, file_name: str):
    os.makedirs("data/model/zipfiles", exist_ok=True)

    try:
        # Faz o download
        response = requests.get(URL)
        response.raise_for_status()  # Lança erro se status != 200
        
        # Salva o arquivo
        file_path = os.path.join("data", "model", "zipfiles", f"{file_name}.zip")
        with open(file_path, "wb") as file:
            file.write(response.content)
        return f"data/model/zipfiles/{file_name}.zip"
    
    except Exception as e:
        print(f"Erro: {str(e)}")


def unzip_file(source_path: str, final_path: str):
    with zipfile.ZipFile(source_path, "r") as zip_ref:
        zip_ref.extractall(final_path)
        print("All unzip!")  # Substitua pelo caminho desejado

#### CBOW

A arquitetura CBOW (Continuous Bag-of-Words) tem como objetivo prever uma palavra central com base no contexto das palavras que a cercam. Ela utiliza mais processamento uma vez que ela necessita analisar as palavras ao redor para que ela busque qual melhor palavra que se encaixa naquele contexto.
Na palavra abaixo:
> Eu vou para a __________ estudar com a professora!

O algoritmo analisa as palavras ao redor da palavra buscada e por exemplo, identifica que estudar tem relação com outras palavras como __casa__, __escola__, __biblioteca,__ dentre outras haver com esse contexto. Quando ele analisa as outras palavras da frase ele vê que __professora__ está mais relacionado com __escola__ do que com __biblioteca__ por exemplo e com base nessa ideia ele indica de maneira probabilística que a melhor palavra indicada para esse caso seja __escola__.

Em geral, algumas __vantagens__ que acompanha esse tipo de técnica é que o CBOW converge mais rapidamente do que o Skip-gram, pois precisa prever uma única palavra central a partir de múltiplos contextos, o que torna o problema de aprendizado um pouco mais fácil. Além disso, ele apresenta boa representação para palavras frequentes já que CBOW tende a aprender boas representações para palavras que aparecem com frequência no corpus, pois se beneficia da agregação de informações de múltiplos contextos. Além disso, ele é menos sensível a palavras raras já que como ele utiliza o contexto para prever a palavra central, o impacto de palavras raras no treinamento geral pode ser menor.

##### Explorando um pouco a arquiterura CBOW

Baixando o modelo CBOW treinado para português brasileiro.

In [6]:
URL_CBOW_50D = "http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s50.zip"

path_final = download_file_zip(URL_CBOW_50D, "cbow_ptbr_50d")
unzip_file(path_final, "data/model/cbow_ptbr_50d")

All unzip!


In [7]:
model_cbow = KeyedVectors.load_word2vec_format('data/model/cbow_ptbr_50d/cbow_s50.txt')

Um exemplo de como o modelo representa as palavras, o termo __carro__, é representado nesse modelo de 50 dimensões com o vetor abaixo:

In [8]:
print(model_cbow['carro'])

[ 0.076797  0.186413  0.101257 -0.028552  0.378992  0.126506  0.068129
  0.084008  0.297164  0.484367 -0.383603  0.106976 -0.733885 -0.177618
  0.013394  0.073068  0.038802 -0.177071  0.068394 -0.138758 -0.263481
  0.668871 -0.11784   0.27917   0.305898 -0.336117  0.002059 -0.078514
  0.015225 -0.095904 -0.009189  0.101624  0.472458  0.414817  0.034765
  0.376102  0.323593 -0.245734 -0.304397  0.666101  0.304205  0.116706
 -0.085813 -0.073047  0.403582  0.391735  0.038815 -0.078577 -0.497931
  0.013595]


Assim como o termo __veículo__ é representado pelo vetor:

In [9]:
print(model_cbow['veículo'])

[ 0.154772  0.166664  0.137601 -0.006604  0.284348  0.012294  0.019751
 -0.052358  0.293641  0.373939 -0.331572  0.184219 -0.863204 -0.037877
  0.121309  0.092585  0.185943 -0.040572  0.005607 -0.28191  -0.407895
  0.707105  0.002234  0.151225  0.285646 -0.385785 -0.008408 -0.06461
  0.003499 -0.121084  0.026264  0.174615  0.467253  0.402033  0.286852
  0.266623  0.344804 -0.057761 -0.18245   0.745378  0.281571  0.051791
 -0.128579  0.087394  0.567206  0.334446  0.03194  -0.169492 -0.454074
  0.025866]


##### Explorando o modelo

Uma vez com esse modelo carregado, pode-se utilizar algumas funções da biblioteca para exploração:

In [10]:
# Encontrar as 10 palavras mais similares do modelo
lista_palavras = model_cbow.most_similar('carro', topn=10)
lista_mais_similares_carro = pd.DataFrame(lista_palavras, columns=['Palavra', 'Similaridade'])
lista_mais_similares_carro.head(10)

,Palavra,Similaridade
0,veículo,0.947191
1,caminhão,0.923407
2,elevador,0.920076
3,avião,0.919352
4,carrinho,0.903016
5,jipe,0.893961
6,passageiro,0.891466
7,cofre,0.887545
8,guindaste,0.881386
9,barco,0.879543


E qual seria a similaridade entre as palavas __mulher__ e __rainha__?

In [11]:
print(f"A similaridade entre as palavras MULHER e RAINHA é {model_cbow.similarity('mulher', 'rainha')}")

A similaridade entre as palavras MULHER e RAINHA é 0.5603883266448975


Descobrindo a palavra dentre um grupo que menos tem relação com as outras do grupo.

In [12]:
lista_palavras = ["maçã", "banana", "laranja", "cachorro"]
palavra_menos_similar = model_cbow.doesnt_match(lista_palavras)
print(f"A palavra do grupo {lista_palavras} que menos está relacionado é {palavra_menos_similar.upper()}!")

A palavra do grupo ['maçã', 'banana', 'laranja', 'cachorro'] que menos está relacionado é CACHORRO!


#### SKIPGRAM

A arquitetura Skip-Gram (Continuous Skip-Gram) tem como objetivo prever as palavras de contexto a partir da palavra central. Ela utiliza mais processamento, pois precisa prever múltiplas palavras de contexto para cada palavra de entrada, calculando probabilidades para cada termo no vocabulário.

Na palavra abaixo:

> Eu vou para a escola estudar com a ______!

O algoritmo toma como entrada a palavra central (por exemplo, estudar) e, a partir dela, tenta prever quais palavras costumam aparecer ao seu redor. Nesse contexto, ele identifica que “estudar” está associado a termos como “casa”, “escola”, “biblioteca”, “professora” etc. Ao processar a frase acima, o Skip-Gram calcula a probabilidade de cada uma dessas palavras aparecer no lugar do blank e, de forma probabilística, indica qual contexto é mais provável naquele ponto (por exemplo, “professora”).

Em geral, algumas vantagens que acompanham esse tipo de técnica são uma melhora para palavras raras uma vez que como o modelo prevê múltiplos contextos para cada palavra central, o Skip-Gram aprende representações de alta qualidade mesmo para termos pouco frequentes no corpus. Além disso, essa arquitetura captura nuances dos diferentes contextos em que uma palavra aparece, pois cada ocorrência contribui para ajustar o vetor central.

##### Explorando um pouco a arquiterura CBOW

Baixando o modelo SKIPGRAM para o português brasileiro.

In [13]:
URL_SKIPGRAM_50D = "http://143.107.183.175:22980/download.php?file=embeddings/word2vec/skip_s50.zip"

path_final = download_file_zip(URL_SKIPGRAM_50D, "skipgram_ptbr_50d")
unzip_file(path_final, "data/model/skipgram_ptbr_50d")

All unzip!


In [14]:
model_skipgram = KeyedVectors.load_word2vec_format("data/model/skipgram_ptbr_50d/skip_s50.txt")

Para os mesmos termos anteriores, agora tem-se a aplicação com a arquitetura SKIPGRAM

In [15]:
print(model_skipgram['carro']) # Vetor que representa o termo 'carro'

[ 1.57448e-01  2.48270e-02  2.42879e-01 -1.15011e-01 -4.18780e-01
 -2.34380e-01  1.74927e-01 -2.32774e-01 -2.74560e-01  2.81151e-01
  2.08446e-01  5.24030e-02 -3.32489e-01  9.69240e-02 -2.27363e-01
 -2.30290e-01 -3.44212e-01  7.46970e-02 -2.64454e-01 -1.95595e-01
  2.64872e-01  5.40117e-01 -6.81250e-02 -3.52761e-01  1.78737e-01
 -5.42237e-01  2.56185e-01  1.32864e-01  8.64210e-02  8.47680e-02
 -1.57026e-01  5.10180e-02  6.77510e-02  4.59000e-03  1.29331e-01
  2.86992e-01  2.21572e-01 -2.70330e-02 -5.73999e-01 -2.26356e-01
  2.99155e-01  3.06765e-01 -3.37014e-01  1.06280e-01 -5.25783e-01
  2.98848e-01  1.65159e-01  5.82686e-01  2.05000e-04  1.00305e-01]


In [16]:
print(model_skipgram['veículo']) # Vetor que representa o termo 'veículo'

[ 0.147363  0.04646   0.180274 -0.334511 -0.523935 -0.165814 -0.120746
 -0.020601 -0.471853  0.329582  0.24704  -0.200887 -0.520996  0.059968
 -0.075358 -0.236901 -0.284175 -0.097785  0.001593 -0.267892  0.158436
  0.497087 -0.022045 -0.602294  0.310654 -0.797712  0.11438  -0.402193
 -0.245227  0.046266  0.052175  0.038362  0.292212 -0.127221 -0.182354
  0.205467 -0.014145  0.313589 -0.382437 -0.13965  -0.076401 -0.126656
 -0.126633  0.376273 -0.51564   0.338938 -0.133791  0.933368 -0.251724
  0.348829]


#### Explorando o Modelo

Agora, com a outra arquitetura, seus valores são diferentes pois seu treinamento foi feito com outra técnica.

In [17]:
# Encontrar as 10 palavras mais similares do modelo
lista_palavras = model_skipgram.most_similar('carro', topn=10)
lista_mais_similares_carro = pd.DataFrame(lista_palavras, columns=['Palavra', 'Similaridade'])
lista_mais_similares_carro.head(10)

,Palavra,Similaridade
0,passageiro,0.907621
1,trator,0.906917
2,caminhão,0.896178
3,jipe,0.889147
4,avião,0.878630
5,guindaste,0.871920
6,contêiner,0.868500
7,parabrisa,0.866084
8,elevador,0.855951
9,motorista,0.848222


Medindo a similaridade entre os termos __mulher__ e __rainha__.

In [18]:
print(f"A similaridade entre as palavras MULHER e RAINHA é {model_skipgram.similarity('mulher', 'rainha')}")

A similaridade entre as palavras MULHER e RAINHA é 0.6356894969940186


A mesma prática para descobrir qual o termo menos relacionado. Nesse caso, indepedentemente da arquitetura, a eficiência se mantém.

In [19]:
lista_palavras = ["maçã", "banana", "laranja", "cachorro"]
palavra_menos_similar = model_skipgram.doesnt_match(lista_palavras)
print(f"A palavra do grupo {lista_palavras} que menos está relacionado é {palavra_menos_similar.upper()}!")

A palavra do grupo ['maçã', 'banana', 'laranja', 'cachorro'] que menos está relacionado é CACHORRO!


### Referências

- [Word2Vec e sua importância na etapa de pré-processamento](https://medium.com/@everton.tomalok/word2vec-e-sua-importância-na-etapa-de-pré-processamento-d0813acfc8ab)
- [Word Embedding: fazendo o computador entender o significado das palavras](https://medium.com/turing-talks/word-embedding-fazendo-o-computador-entender-o-significado-das-palavras-92fe22745057)
- [What Is Word2Vec and How Does It Work?](https://swimm.io/learn/large-language-models/what-is-word2vec-and-how-does-it-work)